In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Orthogroup construction between L. Tarentolae and L. infantum
Author: Dr Thomas Cokelaer, 2025 

- reference infantum: GCA_900500625.2   -- 36 contigs 
- reference tarentolae: GCA_009731335.1    -- 179 contigs

In [26]:
from sequana import *
from pylab import *
from bioconvert import fasta2faa # depends only on python
import pandas as pd
import os

# first, we need to build the orthogroups using the GFF and orthofinder. 

We need to extract the amino acids sequences (instead of DNA)

In [27]:
os.makedirs("analysis/orthodata/", exist_ok=True)

In [33]:
# Bioconvert version 1.1 with _ for stop codon so that sequences are not stop in the middle

In [34]:
if os.path.exists("references/proteins/tarentolae.faa") is False:
    g = GFF3("references/tarentolae.gff")
    g.save_gff_filtered("temp.gff", features = ['gene'])
    g.to_fasta("references/tarentolae.fa", "temp.fa", features=['gene'], identifier="ID")
    conv = fasta2faa.FASTA2FAA("temp.fa", "references/proteins/tarentolae.faa")    
    conv()

if os.path.exists("references/proteins/infantum.faa") is False:
    g = GFF3("references/infantum.gff")
    g.save_gff_filtered("temp.gff", features = ['gene'])
    g.to_fasta("references/infantum.fa", "temp.fa", features=['gene'], identifier="ID")
    from bioconvert import fasta2faa # depends only on python
    conv = fasta2faa.FASTA2FAA("temp.fa", "references/proteins/infantum.faa")
    conv()

if os.path.exists("temp.fa"):
    os.unlink("temp.fa")
if os.path.exists("temp.gff"):
    os.unlink("temp.gff")

try:
    os.symlink("references/proteins/infantum.faa", "analysis/orthodata/infantum.faa")
except:
    pass


try:
    os.symlink("references/proteins/tarentolae.faa", "analysis/orthodata/tarentolae.faa")
except:
    pass



In [35]:
%%bash
# run only once
rm -rf TEMP && orthofinder -f analysis/orthodata/ -o TEMP && cp -r TEMP/Results_*/Orthogroups/ analysis



OrthoFinder version 2.5.5 Copyright (C) 2014 David Emms

2026-01-16 22:08:45 : Starting OrthoFinder 2.5.5
20 thread(s) for highly parallel tasks (BLAST searches etc.)
2 thread(s) for OrthoFinder algorithm

Checking required programs are installed
----------------------------------------
Test can run "mcl -h" - ok
Test can run "fastme -i TEMP/Results_Jan16/WorkingDirectory/dependencies/SimpleTest.phy -o TEMP/Results_Jan16/WorkingDirectory/dependencies/SimpleTest.tre" - ok

Dividing up work for BLAST for parallel processing
--------------------------------------------------
2026-01-16 22:08:45 : Creating diamond database 1 of 2
2026-01-16 22:08:45 : Creating diamond database 2 of 2

Running diamond all-versus-all
------------------------------
Using 20 thread(s)
2026-01-16 22:08:45 : This may take some time....
2026-01-16 22:09:31 : Done all-versus-all sequence search

Running OrthoFinder algorithm
-----------------------------
2026-01-16 22:09:31 : Initial processing of each species
20

# Reading orthogroups and cleanup

In [37]:
from sequana.orthofinder import OrthoGroups
o = OrthoGroups("analysis/Orthogroups/")
o.summary()

The orthogroup contains 7627 groups. Each sample has:
Found 7533 groups in sample infantum in including 8152 genes
Found 7551 groups in sample tarentolae in including 8261 genes

Unassigned genes.
Found 973 unassigned genes in sample Orthogroup
Found 531 unassigned genes in sample infantum
Found 442 unassigned genes in sample tarentolae


In [38]:
ortho_groups = pd.read_csv("analysis/Orthogroups/Orthogroups.tsv", sep="\t")

In [39]:
# number of orthogroups with both species
len(ortho_groups.dropna())

7457

In [40]:
# number of groups where infantum is undefined (tarentolae only)
print(f"Number of groups where infantum is missing : {ortho_groups.isna()['infantum'].sum()}")

# number of groups where tarentolae is undefined (infantum only)
print(f"Number of groups where tarentolae is missing : {ortho_groups.isna()['tarentolae'].sum()}")



Number of groups where infantum is missing : 94
Number of groups where tarentolae is missing : 76


In [41]:
S = sum([len(x.split(",")) for x in ortho_groups[ortho_groups.isna()['infantum']]['tarentolae']])
print(f"Number of tarentolae genes where infantum is missing : {S}")
S = sum([len(x.split(",")) for x in ortho_groups[ortho_groups.isna()['tarentolae']]['infantum']])
print(f"Number of infantum genes where tarentolae is missing : {S}")


Number of tarentolae genes where infantum is missing : 342
Number of infantum genes where tarentolae is missing : 353


In [42]:
# Let us drop the orthogroups with only one species.
ortho_groups = ortho_groups.dropna()

# Some groups are not homogenous.
- case 1: for a given group, one species has several genes that are not on the same chromosome
- case 2: the chromomosome for species 1 is different from species 2

Those cases maybe biological but we want to group orthogroups by chromosome so, let us filter these cases

## cleaning 1: non homogenous chromosomal gene location within a species

In [43]:
# first, remove groups where chromosomes are not homogenous within one of the species
indices_inf = []
indices_tar = []
for index in ortho_groups.index:    
    # focus on infantum first
    x = ortho_groups.loc[index,'infantum']
    # chromosome is contain in the name
    X = set([y.split("_")[-1][0:2] for y in x.split(",")])
    if len(X) != 1:
        indices_inf.append(index)
        #print(f"infantum {index, X}")


    x = ortho_groups.loc[index,'tarentolae']
    # chromosome is contain in the name
    X = set([y.split("_")[-1][0:2] for y in x.split(",")])
    if len(X) != 1:
        indices_tar.append(index)
        #print(f"tarentolae {index, X}")
    

In [44]:
# 47 incoherent groups in infantum, 80 in tarentolae and 115 in total
S1 = set(indices_inf)
S2 = set(indices_tar)
S3 = S1.union(S2)
print(len(S1), len(S2), len(S3))

47 80 115


In [45]:
# remove the groups where one of the species has a heterogenous chromosomal gene locations
ortho_groups = ortho_groups.loc[[x for x in ortho_groups.index if x not in S3]]


In [46]:
# let us add the chromosome of the species for each group
# add chromosome
chr_name_inf = []
chr_name_tar = []
for index in ortho_groups.index:    
    x = ortho_groups.loc[index,'infantum']
    X = set([y.split("_")[-1][0:2] for y in x.split(",")])
    chr_name_inf.append(list(X)[0])

    x = ortho_groups.loc[index,'tarentolae']
    X = set([y.split("_")[-1][0:2] for y in x.split(",")])
    chr_name_tar.append(list(X)[0])
ortho_groups['chr_name_inf'] = chr_name_inf
ortho_groups['chr_name_tar'] = chr_name_tar



# Cleaning 2: remove incoherent chromosome location between the 2 species

In [47]:
indices = []
for index in ortho_groups.index:
    x = ortho_groups.loc[index, 'chr_name_inf']
    y = ortho_groups.loc[index, 'chr_name_tar']
    if x!=y:
        indices.append(index)
print(f"Found {len(indices)} incoherent chrom name")
ortho_groups = ortho_groups.loc[[x for x in ortho_groups.index if x not in indices]]
# 25 incoherent chrom names

Found 25 incoherent chrom name


In [48]:
len(ortho_groups)
# should be 7317 but regression in the notebook

7317

In [49]:
Nfinal = sum([len(x.split(',')) for x in ortho_groups['infantum'].values]) + sum([len(x.split(',')) for x in ortho_groups['tarentolae'].values])

In [50]:
N = len(GFF3("references/tarentolae.gff").df.query("genetic_type=='gene'")) + len(GFF3("references/infantum.gff").df.query("genetic_type=='gene'")) 
print(f"Original orthofinder assignement is {16413 / N * 100}%")
final_assignation = Nfinal / N * 100
print(f"Original orthofinder assignement is {final_assignation}%")

Original orthofinder assignement is 94.4035430806396%
Original orthofinder assignement is 87.36339583572989%


In [53]:
ortho_groups.to_csv("analysis//Orthogroups_cleaned.tsv", sep="\t", index=None)